In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from PIL import Image
import torchvision


from functools import partial

class ImgDataset(Dataset):
    def __init__(self, files, labels=None):
        self.files = files
        self.labels = labels

    def __len__(self):
        return len(self.files)

    def __getitem__(self, index):
        img_path = self.files[index]
        image = Image.open(img_path).convert("L")  # convert to grayscale

        image = torchvision.transforms.ToTensor()(image)

        if self.labels is not None:
            label = self.labels[index]
            return image, label
        else:
            return image

def create_data_loaders(train_files, train_labels, val_files, val_labels, batch_size):
    train_dataset = ImgDataset(train_files, train_labels)
    val_dataset = ImgDataset(val_files, val_labels)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=8, drop_last=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=8, drop_last=True)

    return train_loader, val_loader

class ImgClassifier(nn.Module):
    def __init__(self, num_classes, num_layers, dropout):
        super(ImgClassifier, self).__init__()
        layers = []
        in_channels = 1
        out_channels = 64

        for i in range(num_layers):
            layers.extend([
                nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
                nn.BatchNorm2d(out_channels),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2, stride=2),
                nn.Dropout(dropout),
            ])
            in_channels = out_channels
            out_channels *= 2

        self.conv_layers = nn.Sequential(*layers)
        self.adaptive_avg_pool = nn.AdaptiveAvgPool2d((1, 1))

        # Dummy input to calculate fully connected layer's input size
        dummy_input = torch.randn(1, 1, 128, 128)
        dummy_output = self._forward_conv(dummy_input)
        self.fc_input_size = dummy_output.view(-1).size(0)

        self.fc = nn.Linear(self.fc_input_size, num_classes)

    def _forward_conv(self, x):
        x = self.conv_layers(x)
        x = self.adaptive_avg_pool(x)
        return x

    def forward(self, x):
        x = self._forward_conv(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

def train_model(config, checkpoint_dir=None, data_dir=None, num_epochs=30):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    train_loader, val_loader = create_data_loaders(
        train_files, train_labels, val_files, val_labels,
        config["batch_size"],
    )

    model = ImgClassifier(config["num_classes"], config["num_layers"], config["dropout"]).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=config["lr"])

    if checkpoint_dir:
        model_state, optimizer_state = torch.load(os.path.join(checkpoint_dir, "checkpoint"))
        model.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)

    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device).long()

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader)

        model.eval()
        with torch.no_grad():
            running_corrects = 0
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                running_corrects += torch.sum(preds == labels.data)

            val_accuracy = running_corrects.double() / len(val_loader.dataset)

        tune.report(loss=epoch_loss, accuracy=val_accuracy.cpu().numpy())

        with tune.checkpoint_dir(step=epoch) as checkpoint_dir:
            torch.save((model.state_dict(), optimizer.state_dict()), os.path.join(checkpoint_dir, "checkpoint"))

def get_best_trial_custom(trials, metric_name, mode):
    best_trial = None
    best_metric_value = None

    for trial in trials:
        metric_value = trial.last_result[metric_name].item()
        if best_trial is None or (mode == "max" and metric_value > best_metric_value) or (
            mode == "min" and metric_value < best_metric_value
        ):
            best_trial = trial
            best_metric_value = metric_value

    return best_trial

if __name__ == "__main__":
    import warnings
    warnings.filterwarnings("ignore")

    # Load your audio files and labels here
    import pandas as pd
    from sklearn.preprocessing import LabelEncoder

    #Init Train
    train = pd.read_csv('/home/tech/JupyterLab/train.csv')
    files = ['/home/tech/JupyterLab/train/output' + str(i+1) for i in range(61318)]
    #append .png to each file
    files = [i + '.png' for i in files]
    labels = train['label'].values

    #train test split
    from sklearn.model_selection import train_test_split
    train_files, val_files, train_labels, val_labels = train_test_split(files, labels, test_size=0.2, random_state=2023)

    print("LOADING DATA DONE")

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


    # Set up RayTune experiment
    # config = {
    #     "num_classes": 2,
    #     "lr": tune.loguniform(1e-5, 1e-3),
    #     "batch_size": 16,
    #     "num_layers": 4,
    #     "dropout": tune.uniform(0.1, 0.5),
    # }
    
    #best_config = {'num_classes': 2, 'lr': 0.00024866555826828454, 'batch_size': 16, 'num_layers': 4, 'dropout': 0.35346454212431155}

    
    config = {
        "num_classes": 2,
        "lr": 0.00024866555826828454,
        "batch_size": 16,
        "num_layers": 4,
        "dropout": 0.35346454212431155,
    }

    scheduler = ASHAScheduler(
        metric="accuracy",
        mode="max",
        max_t=30,
        grace_period=1,
        reduction_factor=2
    )

    reporter = CLIReporter(
        metric_columns=["loss", "accuracy", "training_iteration"]
    )

    result = tune.run(
        partial(train_model, data_dir = None),
        resources_per_trial={"cpu": 16, "gpu": 1},
        config=config,
        num_samples=1,
        scheduler=scheduler,
        progress_reporter=reporter,
        name="audio_classification_tune"
    )

    best_trial = get_best_trial_custom(result.trials, "accuracy", "max")
    print(f"Best trial config: {best_trial.config}")
    print(f"Best trial final validation loss: {best_trial.last_result['loss']}")
    print(f"Best trial final validation accuracy: {best_trial.last_result['accuracy']}")

    # Save the best model
    best_trained_model = ImgClassifier(best_trial.config["num_classes"], best_trial.config["num_layers"], best_trial.config["dropout"]).to(device)
    best_trained_model.to(device)
    best_checkpoint_dir = best_trial.checkpoint.dir_or_data
    model_state, optimizer_state = torch.load(os.path.join(best_checkpoint_dir, "checkpoint"))
    best_trained_model.load_state_dict(model_state)
    torch.save(best_trained_model, 'best_model_2.pth')

    # # Save the config used for training
    # import json
    # with open('config_2.json', 'w') as f:
    #     json.dump(best_trial.config, f)


    print("SAVING DONE")

LOADING DATA DONE


E0429 16:44:16.770316299 2952862 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0429 16:44:16.787020119 2952862 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0429 16:44:17.216774025 2952862 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0429 16:44:17.241276833 2952862 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
2023-04-29 16:44:17,260	INFO worker.py:1625 -- Started a local Ray instance.
2023-04-29 16:44:17,899	INFO tune.py:218 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `tune.run(...)`.
/home/tech/.pyenv/versions/3.9.10/lib/python3.9/site-packages/ray/tune/trainable/function_trainable.py:611: DeprecationWarning: `checkpoint_dir` in `func(config, checkpoint_dir)` is being deprecated. To sav

== Status ==
Current time: 2023-04-29 16:44:18 (running for 00:00:00.10)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 16.000: None | Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Logical resource usage: 16.0/16 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /home/tech/ray_results/audio_classification_tune
Number of trials: 1/1 (1 RUNNING)
+-------------------------+----------+-----------------------+
| Trial name              | status   | loc                   |
|-------------------------+----------+-----------------------|
| train_model_07268_00000 | RUNNING  | 10.164.196.15:2954990 |
+-------------------------+----------+-----------------------+




(func pid=2954990) E0429 16:44:21.112627786 2955030 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
(func pid=2954990) E0429 16:44:21.165390782 2955030 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
(func pid=2954990) E0429 16:44:21.199360683 2955030 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
(func pid=2954990) E0429 16:44:21.226198608 2955030 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
(func pid=2954990) E0429 16:44:21.252239813 2955030 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
(func pid=2954990) E0429 16:44:21.278827803 2955030 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
(func pid=2954990) E0429 16:44:21.306025613 2955030 fork_posix.c

== Status ==
Current time: 2023-04-29 16:44:24 (running for 00:00:06.25)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 16.000: None | Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Logical resource usage: 16.0/16 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /home/tech/ray_results/audio_classification_tune
Number of trials: 1/1 (1 RUNNING)
+-------------------------+----------+-----------------------+
| Trial name              | status   | loc                   |
|-------------------------+----------+-----------------------|
| train_model_07268_00000 | RUNNING  | 10.164.196.15:2954990 |
+-------------------------+----------+-----------------------+


== Status ==
Current time: 2023-04-29 16:44:29 (running for 00:00:11.25)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 16.000: None | Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Logical resource usage: 16.0/16 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Resul

...

== Status ==
Current time: 2023-04-29 16:49:02 (running for 00:04:44.97)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 16.000: 0.9793705153294194 | Iter 8.000: 0.9766797129810828 | Iter 4.000: 0.966405740378343 | Iter 2.000: 0.9557240704500978 | Iter 1.000: 0.9413731245923026
Logical resource usage: 16.0/16 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /home/tech/ray_results/audio_classification_tune
Number of trials: 1/1 (1 RUNNING)
+-------------------------+----------+-----------------------+-----------+------------+----------------------+
| Trial name              | status   | loc                   |      loss |   accuracy |   training_iteration |
|-------------------------+----------+-----------------------+-----------+------------+----------------------|
| train_model_07268_00000 | RUNNING  | 10.164.196.15:2954990 | 0.0065534 |   0.978392 |                   28 |
+-------------------------+----------+-----------------------+-----------+------------+------------

(func pid=2954990) E0429 16:49:06.163647270 2955030 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
(func pid=2954990) E0429 16:49:06.187220814 2955030 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
(func pid=2954990) E0429 16:49:06.214920242 2955030 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
(func pid=2954990) E0429 16:49:06.243785061 2955030 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
(func pid=2954990) E0429 16:49:06.273190993 2955030 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
(func pid=2954990) E0429 16:49:06.301685424 2955030 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
(func pid=2954990) E0429 16:49:06.329789395 2955030 fork_posix.c

== Status ==
Current time: 2023-04-29 16:49:12 (running for 00:04:54.40)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 16.000: 0.9793705153294194 | Iter 8.000: 0.9766797129810828 | Iter 4.000: 0.966405740378343 | Iter 2.000: 0.9557240704500978 | Iter 1.000: 0.9413731245923026
Logical resource usage: 16.0/16 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /home/tech/ray_results/audio_classification_tune
Number of trials: 1/1 (1 RUNNING)
+-------------------------+----------+-----------------------+-----------+------------+----------------------+
| Trial name              | status   | loc                   |      loss |   accuracy |   training_iteration |
|-------------------------+----------+-----------------------+-----------+------------+----------------------|
| train_model_07268_00000 | RUNNING  | 10.164.196.15:2954990 | 0.0053452 |   0.979615 |                   29 |
+-------------------------+----------+-----------------------+-----------+------------+------------

(func pid=2954990) E0429 16:49:15.591690211 2955030 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
(func pid=2954990) E0429 16:49:15.615352054 2955030 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
(func pid=2954990) E0429 16:49:15.642901146 2955030 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
(func pid=2954990) E0429 16:49:15.670558157 2955030 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
(func pid=2954990) E0429 16:49:15.696866224 2955030 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
(func pid=2954990) E0429 16:49:15.725167198 2955030 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
(func pid=2954990) E0429 16:49:15.753446955 2955030 fork_posix.c

== Status ==
Current time: 2023-04-29 16:49:16 (running for 00:04:58.81)
Using AsyncHyperBand: num_stopped=1
Bracket: Iter 16.000: 0.9793705153294194 | Iter 8.000: 0.9766797129810828 | Iter 4.000: 0.966405740378343 | Iter 2.000: 0.9557240704500978 | Iter 1.000: 0.9413731245923026
Logical resource usage: 0/16 CPUs, 0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /home/tech/ray_results/audio_classification_tune
Number of trials: 1/1 (1 TERMINATED)
+-------------------------+------------+-----------------------+------------+------------+----------------------+
| Trial name              | status     | loc                   |       loss |   accuracy |   training_iteration |
|-------------------------+------------+-----------------------+------------+------------+----------------------|
| train_model_07268_00000 | TERMINATED | 10.164.196.15:2954990 | 0.00587589 |   0.979697 |                   30 |
+-------------------------+------------+-----------------------+------------+------------

In [3]:
best_config = {'num_classes': 2, 'lr': 0.00024866555826828454, 'batch_size': 16, 'num_layers': 4, 'dropout': 0.35346454212431155}

In [4]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchaudio
import torchaudio.transforms as T
import torch.nn as nn

def upload_CTFSG(token, grader, file):
    import urllib.request, os, json
    urllib.request.urlretrieve('https://raw.githubusercontent.com/alttablabs/ctfsg-utils/master/pyctfsglib.py', './pyctfsglib.py')
    print('Downloaded pyctfsglib.py:', 'pyctfsglib.py' in os.listdir())
    import pyctfsglib as ctfsg
    grader = ctfsg.DSGraderClient(grader, token)
    response = json.loads(grader.submitFile(file))
    os.rename(file, f'{response["multiplier"]}_sklearn_{file[:-4]}.csv')
    return response

if __name__ == "__main__":
    import warnings
    warnings.filterwarnings("ignore")

    # Load your audio files and labels here
    import pandas as pd

    #Init Test
    test = pd.read_csv('/home/tech/JupyterLab/submission.csv')
    test_files = ['/home/tech/JupyterLab/test/output' + str(i+1) for i in range(10821)]
    #append .png to each file
    test_files = [i + '.png' for i in test_files]

    # Load the test dataset
    test_dataset = ImgDataset(test_files)
    test_loader = DataLoader(test_dataset, batch_size=best_config["batch_size"], shuffle=False, num_workers=8)

    # Other config
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Load the best trained model
    best_trained_model = torch.load('best_model_2.pth')
    best_trained_model.eval()

    # Perform predictions on the test dataset
    predictions = []
    with torch.no_grad():
        for inputs in test_loader:
            inputs = inputs.to(device)
            outputs = best_trained_model(inputs)
            _, preds = torch.max(outputs, 1)
            predictions.extend(preds.cpu().numpy())

    # Decode the predictions
    test['label'] = predictions
    from datetime import datetime
    outFile = f'submission_{datetime.now().strftime("%Y%m%d%H%M%S")}.csv'
    test.to_csv(outFile, index=False)

    #Upload
    import random
    GRADER_URL = "http://chals.f.cyberthon23.ctf.sg:42021/"
    token = "XQwqczVjRbNLIQbRNlsPvntYEeYqLuXwjWbhnLIKRpIJUjlfxsYmYglKFnFAeaOp"

    print(upload_CTFSG(token, GRADER_URL, outFile))

E0429 16:51:06.151501277 2952862 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0429 16:51:06.216826013 2952862 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0429 16:51:06.249668617 2952862 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0429 16:51:06.288397557 2952862 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0429 16:51:06.324559592 2952862 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0429 16:51:06.360948249 2952862 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0429 16:51:06.396605938 2952862 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0429 16:51:06.435220597 2952862 fork_pos

Downloaded pyctfsglib.py: True
DSGraderClient: Successfully Connected!
[SERVER] MOTD: CHECK your USER_TOKEN and GRADER_URL HTTP address! I'm Anomalypse @ds-anomalypse-5cb7c6dc6d-ln5nl
ProofOfWork Challenge =>  ('CTFSGRB552e71d3009e6300465c2b1b1d9f37fd', 22)
ProofOfWork Answer Found! =>  2799045
{'challenge': {'name': '[CSIT] Anomalypse'}, 'id': 'clh1qtp9rkici0814bwxbkpa5', 'status': 'PARTIALLY_CORRECT', 'multiplier': 0.883254094030639, 'submittedBy': {'username': 'wakatta'}, 'createdAt': '2023-04-29T08:51:07Z'}
